In [25]:
from horton import *
import numpy as np
from numpy.linalg import norm
from numpy import dot,cross
Methane=IOData.from_file('Methane.xyz')
#print Methane.numbers
#Methane.coordinates*=angstrom

In [19]:
def uhf(mol,basis_set='sto-3g'):
    obasis=get_gobasis(mol.coordinates,mol.numbers,basis_set)

    lf=DenseLinalgFactory(obasis.nbasis)
    #orbital integrals
    olp=obasis.compute_overlap(lf)
    kin=obasis.compute_kinetic(lf)
    na=obasis.compute_nuclear_attraction(mol.coordinates,mol.pseudo_numbers,lf)
    er=obasis.compute_electron_repulsion(lf)
    
    exp_alpha=lf.create_expansion()
    exp_beta=lf.create_expansion()

    guess_core_hamiltonian(olp, kin, na, exp_alpha,exp_beta)
    occ_model=AufbauOccModel(5,5)
    occ_model.assign(exp_alpha, exp_beta)

    external = {'nn': compute_nucnuc(mol.coordinates, mol.pseudo_numbers)}
    terms = [
        UTwoIndexTerm(kin, 'kin'),
        UDirectTerm(er, 'hartree'),
        UExchangeTerm(er, 'x_hf'),
        UTwoIndexTerm(na, 'ne'),
    ]
    ham = UEffHam(terms, external)
    dm_alpha = exp_alpha.to_dm()
    dm_beta = exp_beta.to_dm()
    # - SCF solver
    scf_solver = EDIIS2SCFSolver(1e-5,maxiter=300)
    scf_solver(ham, lf, olp, occ_model, dm_alpha, dm_beta)
    return ham.cache['energy']

In [26]:
uhf(Methane)

Initialized: <horton.gbasis.cext.GOBasis object at 0x7f9cee92b738>
  Number of basis functions         : 9
  Number of normalization constants : 27
  Maximum shell type                : 1
  Center     0 :  S3 S3 P3
  Center     1 :  S3
  Center     2 :  S3
  Center     3 :  S3
  Center     4 :  S3

Performing a core Hamiltonian guess.

Starting restricted closed-shell EDIIS+DIIS-SCF
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Iter         Error        CN         Last nv Method          Energy       Change
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
   0  1.52536e+00                          1      -36.0772803908943
   0  5.16863e-01                          2      -39.5538437115363
   0               0.000e+00    1.0000000  2 E                         0.0000000
   1  9.58034e-02                          3      -39.7209216931104
   1               0.000e+00    1.0000000  3 E

-39.72658194014869